In [ ]:
import base
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rushd as rd
import scipy as sp
import seaborn as sns

# enables concurrent editing of base.py
from importlib import reload
reload(base)

### Load data

Load lenti data for all cell types (`data`)

In [ ]:
base_path = rd.datadir/'instruments'/'data'/'attune'
metadata_path = rd.datadir/'projects'/'miR-iFFL'/'plasmids'
data, quantiles, stats, metadata = base.load_data(base_path, metadata_path, 'lenti')

Load PiggyBac data (`data2`)

In [ ]:
data2, quantiles2, stats2, metadata2 = base.load_data(base_path, metadata_path, 'piggybac')

In [ ]:
ts_label = {'na': 'base', 'NT': 'OL', 'T': 'CL', 'none': '–'}
metadata['ts_label'] = metadata['ts_kind'].replace(ts_label)

# Create dicts to specify colors/markers
metadata_dict = metadata.set_index('construct').to_dict('dict')
main_palette = metadata_dict['color']
main_markers = metadata_dict['markers']

In [ ]:
# Since there is no marker-only condition, save the output expression stats for untransduced cells
baseline_df = data[(data['construct'].isin(['UI','UT'])) | (data['virus_dilution']==0)].groupby(['cell','dox','exp','biorep'])['output'].apply(sp.stats.gmean).rename('output_gmean').reset_index()

### Set up figure

In [ ]:
base_size = base.font_sizes['base_size']
smaller_size = base.font_sizes['smaller_size']

sns.set_style('ticks')
sns.set_context('paper', font_scale=1.0, rc={'font.size': base_size, 'font.family': 'sans-serif', 'font.sans-serif':['Arial']})
plt.rcParams.update({'axes.titlesize': base_size, 'axes.labelsize': base_size, 'xtick.labelsize': smaller_size, 'ytick.labelsize': smaller_size,
                     'pdf.fonttype': 42, 
                     'ytick.major.size': 3, 'xtick.major.size': 3, 'ytick.minor.size': 2, 'ytick.major.pad': 2, 'xtick.major.pad': 2, 
                     'lines.linewidth': 1,
                     'axes.spines.right': False, 'axes.spines.top': False, 'axes.labelpad': 2})

In [ ]:
# Create the overall figure, gridspec, and add subfigure labels
fig = plt.figure(figsize=(6.8504, 1.5*2+1.75))
fig_gridspec = matplotlib.gridspec.GridSpec(3, 6, figure=fig, wspace=0.4, hspace=0.4, 
                                            height_ratios=[1.5,1.75,1.5], width_ratios=[1,0.7,1,1.4,1.2,0.8])
subfigures = {
    'A': fig.add_subfigure(fig_gridspec[0,:]),
    'B': fig.add_subfigure(fig_gridspec[1,:2]),
    'C': fig.add_subfigure(fig_gridspec[1,2:5]),
    'D': fig.add_subfigure(fig_gridspec[1,5]),
    'E': fig.add_subfigure(fig_gridspec[2,:3]),
    'F': fig.add_subfigure(fig_gridspec[2,3:]),
}
for label, subfig in subfigures.items():
    subfig.add_artist(matplotlib.text.Text(x=0, y=1, text=f'{label}', fontsize=base.font_sizes['subpanel_label'], 
                                           fontweight='bold', verticalalignment='top',transform=subfig.transSubfigure))
scatter_kwargs = dict(s=4, jitter=0.2, linewidth=0.5, edgecolor='white')

output_path = rd.rootdir/'output'/'fig_delivery-supp'/'fig_delivery-supp.pdf'
fig.savefig(rd.outfile(output_path))

In [ ]:
subfig = subfigures['B']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.35, bottom=0.55, top=0.35, right=0.15)
axes = subfig.subplots(1,2, gridspec_kw=dict(wspace=0.4))

plot_df = stats[(stats['group'].isin(['base','controller'])) & (stats['cell']=='293T') & (stats['dox']==1000) & (stats['moi']==1)]
scatter_kwargs = dict(s=4, jitter=0.2, linewidth=0.5, edgecolor='white')
xlim = (-0.5, len(plot_df['design'].unique())-0.5)

# stat std
ax = axes[0]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='design', y='output_std', hue='construct', palette=main_palette,
                    legend=False, ax=ax, marker=main_markers[construct], **scatter_kwargs)
ax.set(title='Std.', xlim=xlim, xlabel='design', ylabel='', yscale='log',)

# CV
ax = axes[1]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='design', y='output_variation', hue='construct', palette=main_palette,
                    legend=False, ax=ax, marker=main_markers[construct], **scatter_kwargs)
ax.set(title='CV', xlim=xlim, xlabel='', ylabel='',)

for ax in axes: 
    sns.despine(ax=ax)
    ax.set(xticklabels=[l.get_text() if l.get_text()!='0' else 'base' for l in ax.get_xticklabels()])

fig.savefig(rd.outfile(output_path))

In [ ]:
subfig = subfigures['C']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.35, bottom=0.55, top=0.35, right=0.15)
axes = subfig.subplots(1,3, gridspec_kw=dict(width_ratios=(1,1,1), wspace=0.3))

# lenti 293T controls
group_order = ['base','miR','ts3','ts5']
plot_df = stats[~stats['group'].isin(['controller','marker']) & (stats['cell']=='293T') &
                      (stats['dox']==1000) & (stats['moi']==1)].copy()
plot_df['group'] = plot_df['group'].astype(pd.api.types.CategoricalDtype(categories=group_order, ordered=True))
plot_df.sort_values(['group','ts'], inplace=True)

# shift xticks to add more space between promoter groups
buffer = 0.6
num_groups = 3
xtick_locs = [0, 1+buffer, 2+buffer] + [i+buffer*2 for i in range(3,7)]
construct_loc = {k:v for k,v in zip(plot_df['construct'].unique(), xtick_locs)}
plot_df['construct_loc'] = plot_df['construct'].replace(construct_loc)
metadata['construct_loc'] = metadata['construct'].map(construct_loc)
m = metadata.dropna()
m['construct_loc'] = m['construct_loc'].astype(str)
scatter_kwargs2 = dict(s=4, jitter=0.3, linewidth=0.5, edgecolor='white', native_scale=True)
xlim = (-0.5, plot_df['construct_loc'].max()+0.5)

# adjust markers
m.loc[(m['miR_loc']=='CDS') | (m['ts_loc']=='3\''), 'markers'] = 'D'
m_dict = m.set_index('construct').to_dict('dict')
comb_markers = m_dict['markers']

# make xticklabels
def get_label(df):
    group = df['group'].unique()[0]
    d = df.copy()
    col_map = {'base': 'group', 'miR': 'miR', 'ts3': 'ts', 'ts5': 'ts'}
    d['label'] = d[col_map[group]]
    return d

m = m.groupby('group')[m.columns].apply(get_label).reset_index(drop=True)

# stat gmean
ax = axes[0]
for construct, group in plot_df.groupby('construct', sort=False):
    sns.stripplot(data=group, x='construct_loc', y='output_gmean', hue='construct', palette=main_palette,
                    legend=False, ax=ax, marker=comb_markers[construct], **scatter_kwargs2)
ax.set(title='Mean', xlim=xlim, xlabel='', ylabel='', yscale='log', xticks=xtick_locs,)
baseline = baseline_df.loc[(baseline_df['cell']=='293T') & (baseline_df['dox']==0), 'output_gmean'].mean()
ax.axhline(baseline, color='black', ls=':')

# stat std
ax = axes[1]
for construct, group in plot_df.groupby('construct', sort=False):
    sns.stripplot(data=group, x='construct_loc', y='output_std', hue='construct', palette=main_palette,
                    legend=False, ax=ax, marker=comb_markers[construct], **scatter_kwargs2)
ax.set(title='Std.', xlim=xlim, xlabel='', ylabel='', yscale='log', xticks=xtick_locs,)

# slope
ax = axes[2]
for construct, group in plot_df.groupby('construct', sort=False):
    sns.stripplot(data=group, x='construct_loc', y='slope', hue='construct', palette=main_palette,
                    legend=False, ax=ax, marker=comb_markers[construct], **scatter_kwargs2)
ax.set(title='Slope', xlim=xlim, xlabel='', ylim=(0,1.2), ylabel='', xticks=xtick_locs,
       yticks=[0,0.25,0.5,0.75,1], yticklabels=['0.0','','0.5','','1.0'])

for ax in axes:
    sns.despine(ax=ax)

    # add shaded region for miR-only constructs
    span1 = (xtick_locs[0]+(xtick_locs[1]-xtick_locs[0])/2, xtick_locs[2]+(xtick_locs[3]-xtick_locs[2])/2,)
    ax.axvspan(*span1, color=base.get_light_color(base.colors['gray']), alpha=0.2,)
    labels = dict(zip(m.drop_duplicates('construct_loc')['construct_loc'], m.drop_duplicates('construct_loc')['label']))
    ax.set_xticklabels([labels[l.get_text()].replace('.','-') for l in ax.get_xticklabels()], ha='right', rotation=45)
    ax.yaxis.set_minor_formatter(plt.NullFormatter())

fig.savefig(rd.outfile(output_path))

In [ ]:
subfig = subfigures['D']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.35, bottom=0.55, top=0.35, right=0.05)
axes = subfig.subplots(1,1, gridspec_kw=dict(wspace=0.4))

plot_df = stats[(stats['group'].isin(['base','controller'])) & (stats['cell']=='293T') & (stats['dox']==0) & (stats['moi']==1)]
scatter_kwargs = dict(s=4, jitter=0.2, linewidth=0.5, edgecolor='white')
xlim = (-0.5, len(plot_df['design'].unique())-0.5)

# stat mean
ax = axes
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='design', y='output_std', hue='construct', palette=main_palette,
                    legend=False, ax=ax, marker=main_markers[construct], **scatter_kwargs)
ax.set(title='Mean', xlim=xlim, xlabel='design', ylabel='', yscale='log', ylim=(1e1,1e2))

sns.despine(ax=ax)
ax.set(xticklabels=[l.get_text() if l.get_text()!='0' else 'base' for l in ax.get_xticklabels()])
baseline = baseline_df.loc[(baseline_df['cell']=='293T'), 'output_gmean'].mean()
ax.axhline(baseline, color='black', ls=':')
ax.yaxis.set_minor_formatter(plt.NullFormatter())

fig.savefig(rd.outfile(output_path))

In [ ]:
subfig = subfigures['F']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.35, bottom=0.3, top=0.35, right=0.05)
axes = subfig.subplots(1,4, gridspec_kw=dict(wspace=0.5))

plot_df = stats2
scatter_kwargs = dict(s=4, jitter=0.2, linewidth=0.5, edgecolor='white')
xlim = (-0.5, len(plot_df['ts_kind'].unique())-0.5)

# stat mean
ax = axes[0]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='ts_kind', y='output_gmean', hue='construct', palette=main_palette,
                    legend=False, ax=ax, marker=main_markers[construct], **scatter_kwargs)
ax.set(title='Mean', xlim=xlim, xlabel='', ylabel='', yscale='log',)

# stat std
ax = axes[1]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='ts_kind', y='output_std', hue='construct', palette=main_palette,
                    legend=False, ax=ax, marker=main_markers[construct], **scatter_kwargs)
ax.set(title='Std.', xlim=xlim, xlabel='', ylabel='', yscale='log',)

# slope
ax = axes[2]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='ts_kind', y='slope', hue='construct', palette=main_palette,
                    legend=False, ax=ax, marker=main_markers[construct], **scatter_kwargs)
ax.set(title='Slope', xlim=xlim, xlabel='', ylabel='',)

# CV
ax = axes[3]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='ts_kind', y='output_variation', hue='construct', palette=main_palette,
                    legend=False, ax=ax, marker=main_markers[construct], **scatter_kwargs)
ax.set(title='CV', xlim=xlim, xlabel='', ylabel='',)

for ax in axes: 
    sns.despine(ax=ax)
    ax.set_xticklabels([ts_label[x.get_text()] for x in ax.get_xticklabels()], rotation=45, ha='right',)

fig.savefig(rd.outfile(output_path))

In [ ]:
# Save to OneDrive
fig.savefig(rd.outfile(rd.datadir/'manuscripts'/'2024_miR-iFFL'/'figures'/'links'/'fig_delivery-supp.pdf'))